# Test analyse xgboost à partir de graphe réel 

In [2]:
import networkx as nx
import html
import io
import json
import pandas as pd

Pandas: 2.3.3
Numpy: 1.26.4
NetworkX: 3.4.2
Louvain fonctionne !


In [4]:
import random
np.random.seed(42)
random.seed(42)

In [5]:
def load_graphml_safe(path):
    # 1. Lire le fichier brut
    with open(path, 'r', encoding='utf-8') as f:
        raw_data = f.read()
    
    # 2. Convertir les entités HTML (M&Eacute;XICO -> MÉXICO)
    # Cela évite l'erreur de parsing XML
    clean_data = html.unescape(raw_data)
    
    # 3. Charger dans NetworkX via un flux texte
    G = nx.read_graphml(io.StringIO(clean_data))
    
    print(f"✅ Graphe chargé : {G.number_of_nodes()} nœuds et {G.number_of_edges()} liens.")
    return G

# Utilisation
G_real = load_graphml_safe("outputs/graphs/Airports.graphml")

class GraphEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, set):
            return list(obj)
        return super().default(obj)

def save_graph(G, filename):
    data = nx.node_link_data(G)
    with open(filename, 'w') as f:
        # On utilise notre encodeur personnalisé ici
        json.dump(data, f, cls=GraphEncoder)
    print(f"Graphe sauvegardé dans {filename}")

save_graph(G_real, "outputs/graphs/Airports.json")

✅ Graphe chargé : 3363 nœuds et 13547 liens.
Graphe sauvegardé dans outputs/graphs/Airports.json


/Users/bgdu69/miniconda3/envs/stageM2/lib/python3.10/site-packages/networkx/readwrite/json_graph/node_link.py:142: FutureWarning: 
The default value will be `edges="edges" in NetworkX 3.6.

To make this warning go away, explicitly set the edges kwarg, e.g.:

  nx.node_link_data(G, edges="links") to preserve current behavior, or
  nx.node_link_data(G, edges="edges") for forward compatibility.
  warnings.warn(


## 0 - Explo des attributs

In [ ]:
sample_node = list(G_real.nodes(data=True))[0]
print("\nSample node attributes:")
print(sample_node)

## 1 - Calcul des attributs de noeuds et de paires de noeuds

In [87]:
def get_topology_features(G, u, v, precomputed, is_existing_edge=False):
    """Calcule les métriques topologiques pour une paire (u, v)"""
    # 1. Métriques de paires (Voisinage)
    aa = next(nx.adamic_adar_index(G, [(u, v)]))[2]
    jc = next(nx.jaccard_coefficient(G, [(u, v)]))[2]
    pa = next(nx.preferential_attachment(G, [(u, v)]))[2]
    cn = len(list(nx.common_neighbors(G, u, v)))

    # 2. Métriques de Nœuds (extraites du dictionnaire pré-calculé)
    # On ajoute les versions pour u et pour v
    node_features = {
        'pr_u': precomputed['pr'].get(u, 0), 'pr_v': precomputed['pr'].get(v, 0),
        'lcc_u': precomputed['lcc'].get(u, 0), 'lcc_v': precomputed['lcc'].get(v, 0),
        'and_u': precomputed['and'].get(u, 0), 'and_v': precomputed['and'].get(v, 0),
        'dc_u': precomputed['dc'].get(u, 0), 'dc_v': precomputed['dc'].get(v, 0)
    }

    # 3. Shortest Path (SP)
    if is_existing_edge:
        G.remove_edge(u, v)
        try:
            sp = nx.shortest_path_length(G, source=u, target=v)
        except nx.NetworkXNoPath:
            sp = 0 
        G.add_edge(u, v)
    else:
        try:
            sp = nx.shortest_path_length(G, source=u, target=v)
        except nx.NetworkXNoPath:
            sp = 0

    # Fusion de toutes les métriques
    topo_res = {'cn': cn, 'aa': aa, 'jc': jc, 'pa': pa, 'sp': sp}
    topo_res.update(node_features)
    
    return topo_res

def prepare_balanced_data_unknown_pos_and_community(G, negative_ratio=1.0):
    edges = list(G.edges())
    nodes = list(G.nodes())
    n_pos = len(edges)
    data = []
    random.seed(42)

    # --- ÉTAPE CRUCIALE : PRÉ-CALCUL ---
    # On calcule les métriques globales une seule fois ici
    print("Pré-calcul des métriques de nœuds...")
    precomputed = {
        'pr': nx.pagerank(G),                    # PageRank (PR)
        'lcc': nx.clustering(G),                # Local Clustering Coefficient (LCC)
        'and': nx.average_neighbor_degree(G),   # Average Neighbor Degree (AND)
        'dc': nx.degree_centrality(G)           # Degree Centrality (DC)
    }
    
    # --- 1. CLASSE POSITIVE ---
    for u, v in edges:
        topo = get_topology_features(G, u, v, precomputed, is_existing_edge=True)
        
        row = {
            'u': u, 
            'v': v,
            'target': 1
        }
        row.update(topo)
        data.append(row)
    
    # --- 2. CLASSE NÉGATIVE ---
    n_neg_target = int(n_pos * negative_ratio)
    neg_count = 0
    while neg_count < n_neg_target:
        u, v = random.sample(nodes, 2)
        if not G.has_edge(u, v) and u != v:
            topo = get_topology_features(G, u, v, precomputed, is_existing_edge=False)
            
            row = {
                'u': u, 
                'v': v,
                'target': 0
            }
            row.update(topo)
            data.append(row)
            neg_count += 1

    print(f"DataFrame créé <3 : {len(data)} paires de noeuds choisies")
    return pd.DataFrame(data)

In [88]:
heuristics_only_data = prepare_balanced_data_unknown_pos_and_community(G_real)
heuristics_only_data.to_parquet("outputs/datasets/Airports_heuristics_only_dataset.parquet", engine='pyarrow')

Pré-calcul des métriques de nœuds...
DataFrame créé <3 : 27094 paires de noeuds choisies


## 2 - Inférences sur Position & Communities

#### DeepWalk

In [ ]:
print(heuristics_only_data.shape)

#### Node2Vec

In [ ]:
from node2vec import Node2Vec
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
def compute_node2vec_features(G, dimensions=64):
    """
    Génère les embeddings Node2Vec et retourne un dictionnaire {node_id: vector}
    """
    print(f"🚀 Génération des marches aléatoires (dim={dimensions})...")
    
    # Configuration de Node2Vec
    # p=1, q=1 -> équivalent à DeepWalk
    # p=1, q=2 -> Favorise l'exploration locale (structure)
    # p=2, q=0.5 -> Favorise l'exploration lointaine (communautés) - homophilie
    node2vec = Node2Vec(G, 
                        dimensions=dimensions, 
                        walk_length=30, 
                        num_walks=100, 
                        workers=4, 
                        p=1, q=1)

    print("🧠 Entraînement du modèle Skip-gram...")
    model = node2vec.fit(window=10, min_count=1, batch_words=4)
    
    # On récupère les vecteurs dans un dictionnaire
    embeddings = {str(node): model.wv[str(node)] for node in G.nodes()}
    return embeddings

def add_node2vec_to_df(df, embeddings):
    """
    Ajoute des features de similarité basées sur Node2Vec au DataFrame
    """
    print("📊 Calcul des distances vectorielles pour chaque paire...")
    
    def get_cosine_sim(u, v):
        vec_u = embeddings[str(u)].reshape(1, -1)
        vec_v = embeddings[str(v)].reshape(1, -1)
        return cosine_similarity(vec_u, vec_v)[0][0]

    def get_l2_dist(u, v):
        vec_u = embeddings[str(u)]
        vec_v = embeddings[str(v)]
        return np.linalg.norm(vec_u - vec_v)

    # Similarité Cosinus : 1 = très proche, 0 = perpendiculaire
    df['n2v_cosine'] = df.apply(lambda row: get_cosine_sim(row['u'], row['v']), axis=1)
    
    # Distance Euclidienne : plus c'est petit, plus ils sont proches
    df['n2v_dist'] = df.apply(lambda row: get_l2_dist(row['u'], row['v']), axis=1)
    
    return df

# --- UTILISATION ---
# 1. Calcul des vecteurs d'embedding
n2v_embeddings = compute_node2vec_features(G_real)

# 2. Injection dans le DataFrame de training/test
node2vec_homophilie_p2_q0_5_data = add_node2vec_to_df(heuristics_only_data, n2v_embeddings)
node2vec_homophilie_p2_q0_5_data.to_parquet(
    "outputs/datasets/Airports_deepwalk_p1_q1_dataset.parquet", 
    engine='pyarrow'
)

#### Role2Vec Tentative

In [ ]:
from karateclub import Role2Vec
import networkx as nx

In [ ]:
# Karate Club nécessite que les nœuds soient indexés par des entiers de 0 à N-1
mapping = {node: i for i, node in enumerate(G_real.nodes())}
reverse_mapping = {i: node for node, i in mapping.items()}
G_reindexed = nx.relabel_nodes(G_real, mapping)

# 2. Entraînement de Role2Vec
print("🎭 Calcul de Role2Vec (Roles structurels)...")
# On reste sur des dimensions cohérentes avec tes autres tests
model_r2v = Role2Vec(dimensions=64, walk_number=10, walk_length=80) 
model_r2v.fit(G_reindexed)

# 3. Extraction des vecteurs
r2v_embeddings_raw = model_r2v.get_embedding()
r2v_embeddings = {reverse_mapping[i]: r2v_embeddings_raw[i] for i in range(len(G_real))}

print("✅ Role2Vec terminé !")

role2vec_data = add_node2vec_to_df(heuristics_only_data, r2v_embeddings)
role2vec_data.to_parquet(
    "outputs/datasets/Airports_role2vec_dataset.parquet", 
    engine='pyarrow'
)

#### Louvain Communities

In [7]:
import networkx as nx
from networkx.algorithms.community import louvain_communities

In [61]:
communities = nx.community.louvain_communities(G_real, seed=42)

node_to_community = {} 
for i, community in enumerate(communities):
    for node in community:
        node_to_community[node] = i
        
print(len(node_to_community))

heuristics_only_data = pd.read_parquet("outputs/datasets/Airports_heuristics_only_dataset.parquet")
print(type(heuristics_only_data))
louvain_communities_data = heuristics_only_data.copy()

louvain_communities_data["community_u"] = louvain_communities_data["u"].map(node_to_community)
louvain_communities_data["community_v"] = louvain_communities_data["v"].map(node_to_community)
louvain_communities_data["same_community"] = (louvain_communities_data["community_u"] == louvain_communities_data["community_v"]).astype(int)

louvain_communities_data.to_parquet(
    "outputs/datasets/Airports_louvain_communities_dataset.parquet", 
    engine='pyarrow'
)

3363
<class 'pandas.core.frame.DataFrame'>


#### Infomap

In [62]:
from infomap import Infomap

In [77]:
def get_infomap_communities(G):
    im = Infomap("--two-level --silent")
    
    for source, target in G.edges():
        im.add_link(int(source), int(target))
    
    # Exécution
    im.run()
    
    # Extraction des résultats
    # node.node_id est l'ID d'origine, node.module_id est le cluster
    node_to_infomap = {node.node_id: node.module_id for node in im.tree if node.is_leaf}
    return node_to_infomap

# Utilisation
infomap_dict = get_infomap_communities(G_real)
print(f"Infomap a trouvé {len(set(infomap_dict.values()))} communautés.")

infomap_data = heuristics_only_data.copy()

# 2. Mapping avec le dictionnaire Infomap
infomap_data["u"] = pd.to_numeric(infomap_data["u"], errors='coerce').astype(int)
infomap_data["v"] = pd.to_numeric(infomap_data["v"], errors='coerce').astype(int)

infomap_data["infomap_u"] = infomap_data["u"].map(infomap_dict)
infomap_data["infomap_v"] = infomap_data["v"].map(infomap_dict)
infomap_data["same_infomap"] = (infomap_data["infomap_u"] == infomap_data["infomap_v"]).astype(int)

print(type(infomap_dict))
print(infomap_data.head(3))

print(f"Type clé dico: {type(list(infomap_dict.keys())[0])}")

# Vérifie le type de la colonne du DataFrame
print(f"Type colonne DF: {infomap_data['u'].dtype}")
# 4. Sauvegarde
infomap_data.to_parquet("outputs/datasets/Airports_infomap_dataset.parquet", engine='pyarrow')

Infomap a trouvé 179 communautés.
<class 'dict'>
   u  v  target  cn        aa        jc   pa  sp      pr_u      pr_v  lcc_u  \
0  0  1       1   1  0.303413  0.200000    8   2  0.000156  0.000293    1.0   
1  0  2       1   1  0.721348  0.035714   54   2  0.000156  0.001552    1.0   
2  1  2       1   3  4.328085  0.107143  108   2  0.000293  0.001552    0.5   

      lcc_v  and_u      and_v      dc_u      dc_v  infomap_u  infomap_v  \
0  0.500000  15.50   8.250000  0.000595  0.001190         48         48   
1  0.096866  15.50  18.962963  0.000595  0.008031         48         48   
2  0.096866   8.25  18.962963  0.001190  0.008031         48         48   

   same_infomap  
0             1  
1             1  
2             1  
Type clé dico: <class 'int'>
Type colonne DF: int64


## 3 - Comparaison des perfs sur XGBoost

In [78]:
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import f1_score, confusion_matrix, roc_auc_score, average_precision_score
import pandas as pd
import numpy as np

In [81]:
# Chargement des datasets au format Parquet
heuristics_only_data = pd.read_parquet("outputs/datasets/Airports_heuristics_only_dataset.parquet")
deepwalk_data = pd.read_parquet("outputs/datasets/Airports_deepwalk_p1_q1_dataset.parquet")
node2vec_homophilie_p2_q0_5_data = pd.read_parquet("outputs/datasets/Airports_node2vec_homophilie_p2_q0_5dataset.parquet")
node2vec_structural_p1_q2_data = pd.read_parquet("outputs/datasets/Airports_node2vec_structural_p1_q2dataset.parquet")
role2vec_data = pd.read_parquet("outputs/datasets/Airports_role2vec_dataset.parquet")

louvain_communities_data = pd.read_parquet("outputs/datasets/Airports_louvain_communities_dataset.parquet")
infomap_data = pd.read_parquet("outputs/datasets/Airports_infomap_dataset.parquet")

homo_and_louvain_data = node2vec_homophilie_p2_q0_5_data.copy()

homo_and_louvain_data["community_u"] = homo_and_louvain_data["u"].map(node_to_community)
homo_and_louvain_data["community_v"] = homo_and_louvain_data["v"].map(node_to_community)
homo_and_louvain_data["same_community"] = (homo_and_louvain_data["community_u"] == homo_and_louvain_data["community_v"]).astype(int)

homo_and_structural_data = node2vec_homophilie_p2_q0_5_data.copy()
homo_and_structural_data["n2v_cosine_s"] = node2vec_structural_p1_q2_data["n2v_cosine"]
homo_and_structural_data["n2v_dist_s"] = node2vec_structural_p1_q2_data["n2v_dist"]


print("✅ Tous les datasets ont été chargés avec succès depuis le format Parquet.")

# --- CONFIGURATION ---
my_bench_datasets = {
    "Heuristics Only": heuristics_only_data,
    "DeepWalk": deepwalk_data,
    "N2V Homophilie": node2vec_homophilie_p2_q0_5_data,
    "N2V Structural": node2vec_structural_p1_q2_data,
    "Role2Vec": role2vec_data,
    "Louvain Communities": louvain_communities_data,
    "Infomap Communities": infomap_data,
    "Homophilie and Louvain" : homo_and_louvain_data,
    "Homophilie & Structural" : homo_and_structural_data
}

for name, dataset in my_bench_datasets.items():
    print(f"\n--- Dataset: {name} ---")
    print(dataset.head(5))

✅ Tous les datasets ont été chargés avec succès depuis le format Parquet.

--- Dataset: Heuristics Only ---
   u  v  target  cn        aa        jc   pa  sp      pr_u      pr_v  lcc_u  \
0  0  1       1   1  0.303413  0.200000    8   2  0.000156  0.000293    1.0   
1  0  2       1   1  0.721348  0.035714   54   2  0.000156  0.001552    1.0   
2  1  2       1   3  4.328085  0.107143  108   2  0.000293  0.001552    0.5   
3  1  3       1   1  0.303413  0.200000    8   2  0.000293  0.000156    0.5   
4  1  4       1   1  0.303413  0.200000    8   2  0.000293  0.000156    0.5   

      lcc_v  and_u      and_v      dc_u      dc_v  
0  0.500000  15.50   8.250000  0.000595  0.001190  
1  0.096866  15.50  18.962963  0.000595  0.008031  
2  0.096866   8.25  18.962963  0.001190  0.008031  
3  1.000000   8.25  15.500000  0.001190  0.000595  
4  1.000000   8.25  15.500000  0.001190  0.000595  

--- Dataset: DeepWalk ---
   u  v  target  cn        aa        jc   pa  sp      pr_u      pr_v  lcc_u  \

In [90]:
from sklearn.metrics import f1_score, confusion_matrix, roc_auc_score, average_precision_score

def run_benchmark(datasets_dict, K=50):
    results = []
    
    # Colonnes techniques à exclure
    exclude = {'u', 'v', 'target'}
    
    for name, df in datasets_dict.items():
        print(f"Traitement de {name}...")
        
        features = [c for c in df.columns if c not in exclude]
        df_tmp = df.dropna(subset=features + ['target'])
        X = df_tmp[features]
        y = df_tmp['target']
        
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.2, random_state=42, stratify=y
        )
        
        model = XGBClassifier(
            n_estimators=100,
            learning_rate=0.1,
            max_depth=6,
            eval_metric='logloss',
            tree_method='hist',
            n_jobs=-1
        )
        
        model.fit(X_train, y_train)
        
        # Prédictions
        probs = model.predict_proba(X_test)[:, 1]
        preds = model.predict(X_test)  # Seuil par défaut à 0.5
        
        # Métriques de performance
        auc_roc = roc_auc_score(y_test, probs)
        f1 = f1_score(y_test, preds)
        
        # Matrice de confusion
        tn, fp, fn, tp = confusion_matrix(y_test, preds).ravel()
        
        # Hits@K
        top_k_indices = np.argsort(probs)[-K:]
        hits_at_k = y_test.iloc[top_k_indices].sum() / K
        
        results.append({
            'Dataset': name,
            'AUC-ROC': round(auc_roc, 4),
            'F1-Score': round(f1, 4),
            'VP (True Pos)': tp,
            'VN (True Neg)': tn,
            'FP (False Pos)': fp,
            'FN (False Neg)': fn,
            f'Hits@{K}': round(hits_at_k, 4)
        })

    return pd.DataFrame(results).sort_values(by='AUC-ROC', ascending=False)

# --- EXECUTION ---
summary = run_benchmark(my_bench_datasets)
print("\n RÉSULTATS COMPARATIFS")
print(summary.to_string(index=False))

Traitement de Heuristics Only...
Traitement de DeepWalk...
Traitement de N2V Homophilie...
Traitement de N2V Structural...
Traitement de Role2Vec...
Traitement de Louvain Communities...
Traitement de Infomap Communities...
Traitement de Homophilie and Louvain...
Traitement de Homophilie & Structural...

 RÉSULTATS COMPARATIFS
                Dataset  AUC-ROC  F1-Score  VP (True Pos)  VN (True Neg)  FP (False Pos)  FN (False Neg)  Hits@50
               DeepWalk   0.9999    0.9978           2709           2698              12               0      1.0
         N2V Homophilie   0.9999    0.9980           2709           2699              11               0      1.0
 Homophilie and Louvain   0.9999    0.9980           2709           2699              11               0      1.0
Homophilie & Structural   0.9999    0.9980           2709           2699              11               0      1.0
         N2V Structural   0.9998    0.9976           2709           2697              13              

## 4 - Analyse SHAP rapide

In [ ]:
def analyze_with_shap(model, X_test, output_dir="outputs/plots"):
    """Calcule les SHAP values et génère les plots globaux proprement."""
    # 1. Configuration de l'explainer 'Boîte Noire' (le plus stable sur mon Mac)
    # On définit la fonction de prédiction (proba de la classe 1)
    model_predict = lambda x: model.predict_proba(x)[:, 1]
    
    # Utilisation d'un masker (échantillon de référence)
    # On prend 50 lignes pour équilibrer vitesse et précision
    masker = X_test.iloc[:50]
    
    # Initialisation de l'explainer
    explainer = shap.Explainer(model_predict, masker)    
    
    # 2. Calcul effectif des SHAP values
    # On récupère l'objet 'Explanation' complet
    shap_explanation = explainer(X_test)
    
    # 3. Extraction des valeurs numériques pour le retour de fonction
    # On récupère les valeurs brutes (.values)
    shap_values = shap_explanation.values

    # Gestion de la dimension (si SHAP renvoie [n_samples, n_features, 2])
    if len(shap_values.shape) == 3:
        shap_values = shap_values[:, :, 1]
    
    # --- GÉNÉRATION DES PLOTS ---
    os.makedirs(output_dir, exist_ok=True)

    # Plot 1: Summary Points (Beeswarm)
    plt.figure(figsize=(12, 8))
    # On peut passer l'objet explanation directement, c'est plus moderne
    shap.plots.beeswarm(shap_explanation, show=False)
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, "shap_summary_points.png"))
    plt.close()

    # Plot 2: Summary Bar
    plt.figure(figsize=(12, 8))
    shap.plots.bar(shap_explanation, show=False)
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, "shap_summary_bar.png"))
    plt.close()
    
    return shap_values

def calculate_feature_rankings(shap_values, feature_names, output_dir="outputs/plots"):
    """Calcule la distribution des rangs et génère le barplot du Top 5."""
    abs_shap = np.abs(shap_values)
    ranks = np.argsort(-abs_shap, axis=1)
    
    ranking_stats = {}
    n_samples, n_features = shap_values.shape

    for i, name in enumerate(feature_names):
        feature_ranks = np.where(ranks == i)[1] + 1
        counts = np.bincount(feature_ranks, minlength=n_features + 1)[1:]
        ranking_stats[name] = (counts / n_samples) * 100

    df_ranks = pd.DataFrame(ranking_stats, index=[f"Rang {i+1}" for i in range(n_features)])
    
    # Plot 3: Top 5 Appearance
    top5 = df_ranks.iloc[0:5, :].sum(axis=0).sort_values(ascending=False)
    plt.figure(figsize=(12, 7))
    sns.barplot(x=top5.index, y=top5.values, palette="viridis")
    plt.title("Importance structurelle : % de présence dans le Top 5 SHAP")
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, "shap_top5_frequency.png"))
    plt.close()
    
    return df_ranks